In [1]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

data = pd.read_csv('/Users/baby_cow/Desktop/AI/ML_competition/data/geocoded_addresses_12.csv')
data.columns

Index(['Unnamed: 0', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
       '도로명', '거래유형', 'k-전체동수', 'k-전체세대수', 'k-연면적', 'k-주거전용면적', 'k-관리비부과면적',
       'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하',
       'k-135㎡초과', '건축면적', '주차대수', 'target', 'is_test', 'latitude',
       'longitude', 'address', 'address_apt', 'address_apt_sqm', '구', '동',
       '강남여부', 'mean_target', 'apt_mean_target', 'address_mean_target',
       'mean_target_gu_dong', 'closest_bus_stop_distance',
       'closest_subway_station_distance', 'bus_stop_count_500m',
       'subway_station_count_500m', 'bus_stop_count_750m',
       'subway_station_count_750m', 'bus_stop_count_1000m',
       'subway_station_count_1000m', 'square_meter',
       'monthly_mean_target_rate', 'adjusted_square_meter', 'is_outlier',
       'year', 'month', 'distance_to_lead_house'],
      dtype='object')

In [2]:
temp_index = list(range(len(data)))
data['id'] = temp_index

In [3]:
selected_columns = ['id', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '구', '동', 
                    'closest_bus_stop_distance','closest_subway_station_distance', 'bus_stop_count_500m',
                    'subway_station_count_500m', 'bus_stop_count_750m',
                    'subway_station_count_750m', 'bus_stop_count_1000m',
                    'subway_station_count_1000m', 'distance_to_lead_house', 'is_test', 'target']

df = data[selected_columns]
df.head()


,id,아파트명,전용면적(㎡),계약년월,계약일,층,구,동,closest_bus_stop_distance,closest_subway_station_distance,bus_stop_count_500m,subway_station_count_500m,bus_stop_count_750m,subway_station_count_750m,bus_stop_count_1000m,subway_station_count_1000m,distance_to_lead_house,is_test,target
0,0,개포6차우성,79.97,2017-12-01,8,3,강남구,개포동,95.560091,1187.188361,12,0,38,0,57,0,6.609733,0,124000.0
1,1,개포6차우성,79.97,2017-12-01,22,4,강남구,개포동,95.560091,1187.188361,12,0,38,0,57,0,6.609733,0,123500.0
2,2,개포6차우성,54.98,2017-12-01,28,5,강남구,개포동,95.560091,1187.188361,12,0,38,0,57,0,6.609733,0,91500.0
3,3,개포6차우성,79.97,2018-01-01,3,4,강남구,개포동,95.560091,1187.188361,12,0,38,0,57,0,6.609733,0,130000.0
4,4,개포6차우성,79.97,2018-01-01,8,2,강남구,개포동,95.560091,1187.188361,12,0,38,0,57,0,6.609733,0,117000.0


In [4]:
chunks = {}

df['지역'] = df.apply(lambda x: x['구']+' '+ x['동'], axis=1)

regions = list(df['지역'].unique())

for region in regions:
    chunk = df[df['지역'] == region]
    chunk['계약년'] = df['계약년월'].map(lambda x : int(str(x)[:4]))
    chunk.drop(columns=['구', '동'], axis=1, inplace=True)
    chunks[region] = chunk.sort_values(by='계약년월')


In [5]:
from IPython.display import display
from tqdm import tqdm
import pandas as pd

for region in tqdm(regions):
    chunk = chunks[region]

    # 테스트 데이터 제외
    train_data = chunk[chunk['is_test'] == 0]

    # 1. 올해 이 지역 평균
    yearly_mean = train_data.groupby('계약년')['target'].mean()
    chunk['올해 이 지역 평균'] = chunk['계약년'].map(yearly_mean)

    # 2. 올해 이 아파트 평균
    apart_mean = train_data.groupby(['계약년', '아파트명'])['target'].mean()
    chunk['올해 이 아파트 평균'] = chunk.set_index(['계약년', '아파트명']).index.map(apart_mean)

    # 3. 올해 해당 집 유형들의 평균
    type_mean = train_data.groupby(['계약년', '아파트명', '전용면적(㎡)'])['target'].mean()
    chunk['올해 해당 집 유형들의 평균'] = chunk.set_index(['계약년', '아파트명', '전용면적(㎡)']).index.map(type_mean)

    # 4. 평균 부정확도
    chunk['평균 부정확도'] = 0

    chunks[region] = chunk

  0%|          | 0/339 [00:00<?, ?it/s]

100%|██████████| 339/339 [00:04<00:00, 77.39it/s] 


In [6]:
import pandas as pd
import numpy as np
from tqdm import tqdm

def fill_na(sample):
    region_mean = sample['올해 이 지역 평균']
    apart_mean = sample['올해 이 아파트 평균']
    house_mean = sample['올해 해당 집 유형들의 평균']

    if pd.isnull(apart_mean):
        sample['올해 이 아파트 평균'] = region_mean
        sample['평균 부정확도'] = 2

    if pd.isnull(house_mean):
        if pd.isnull(apart_mean):
            sample['올해 해당 집 유형들의 평균'] = region_mean
            sample['평균 부정확도'] = 3
        else:
            sample['올해 해당 집 유형들의 평균'] = apart_mean
            sample['평균 부정확도'] = 1

    return sample

for region in tqdm(regions):
    chunk = chunks[region]
    chunk = chunk.apply(fill_na, axis=1)
    chunks[region] = chunk

100%|██████████| 339/339 [00:39<00:00,  8.66it/s]


In [7]:
null_regions = []
cnt = 0
for region in regions:
    select_cols = [col for col in chunks[region].columns if col != 'target']

    if sum(chunks[region][select_cols].isnull().sum()) != 0:
        null_regions.append(region)
        print(region)
        print(chunks[region][select_cols].isnull().sum())
        print('-' * 100)
        cnt += 1

print(cnt)

성동구 홍익동
id                                 0
아파트명                               0
전용면적(㎡)                            0
계약년월                               0
계약일                                0
층                                  0
closest_bus_stop_distance          0
closest_subway_station_distance    0
bus_stop_count_500m                0
subway_station_count_500m          0
bus_stop_count_750m                0
subway_station_count_750m          0
bus_stop_count_1000m               0
subway_station_count_1000m         0
distance_to_lead_house             0
is_test                            0
지역                                 0
계약년                                0
올해 이 지역 평균                         2
올해 이 아파트 평균                        2
올해 해당 집 유형들의 평균                    2
평균 부정확도                            0
dtype: int64
----------------------------------------------------------------------------------------------------
성북구 동선동4가
id                                 0
아파트명             

In [8]:
# 두번째 결측치 처리
second_null_regions = []

for null_region in null_regions:
    temp_df = chunks[null_region]

    for idx, row in temp_df[temp_df.isnull().any(axis=1)].iterrows():

        year = 2022
        penalty = 4
        apart_name = row['아파트명']
        apart_size = row['전용면적(㎡)']

        region_target_mean = temp_df.loc[(temp_df['is_test'] == 0) & (temp_df['계약년'] == year) , '올해 이 지역 평균'].mean()
        apart_target_mean = temp_df.loc[(temp_df['is_test'] == 0) & (temp_df['계약년'] == year) & (temp_df['아파트명'] == apart_name), '올해 이 아파트 평균'].mean()
        house_target_mean = temp_df.loc[(temp_df['is_test'] == 0) & (temp_df['계약년'] == year) & (temp_df['아파트명'] == apart_name) & (temp_df['전용면적(㎡)'] == apart_size), '올해 해당 집 유형들의 평균'].mean()

        while pd.isnull(region_target_mean):
            year -= 1
            region_target_mean = temp_df.loc[(temp_df['is_test'] == 0) & (temp_df['계약년'] == year) , '올해 이 지역 평균'].mean()
            apart_target_mean = temp_df.loc[(temp_df['is_test'] == 0) & (temp_df['계약년'] == year) & (temp_df['아파트명'] == apart_name), '올해 이 아파트 평균'].mean()
            house_target_mean = temp_df.loc[(temp_df['is_test'] == 0) & (temp_df['계약년'] == year) & (temp_df['아파트명'] == apart_name) & (temp_df['전용면적(㎡)'] == apart_size), '올해 해당 집 유형들의 평균'].mean()
            penalty += 4

        temp_df.loc[idx, '올해 이 지역 평균'] = region_target_mean
        temp_df.loc[idx, '올해 이 아파트 평균'] = apart_target_mean 
        temp_df.loc[idx, '올해 해당 집 유형들의 평균'] = house_target_mean
        temp_df.loc[idx, '평균 부정확도'] = penalty

        if pd.isnull(apart_target_mean):
            temp_df.loc[idx, '올해 이 아파트 평균'] = region_target_mean
            temp_df.loc[idx, '평균 부정확도'] = penalty + 1
        
        if pd.isnull(house_target_mean):
            if pd.isnull(apart_target_mean):
                temp_df.loc[idx, '올해 해당 집 유형들의 평균'] = region_target_mean
                temp_df.loc[idx, '평균 부정확도'] = penalty + 3
            else:
                temp_df.loc[idx, '올해 해당 집 유형들의 평균'] = apart_target_mean
                temp_df.loc[idx, '평균 부정확도'] = penalty + 2

    chunks[null_region] = temp_df

# 학습하고 예측해보자!!

In [12]:
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

result = pd.DataFrame()

result_board = []

params = {
    'objective': 'regression', 
    'metric': 'rmse',           
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.001,
    'feature_fraction': 0.9,
    'lambda_l2' : 0.1,
    'drop_rate' : 0.2,
    'verbose' : -1
}

regions_have_empty_test = []

for region in tqdm(regions):
    chunk = chunks[region]

    select_cols = ['id', '전용면적(㎡)', '층',
        'closest_bus_stop_distance', 'closest_subway_station_distance',
        'bus_stop_count_500m', 'subway_station_count_500m',
        'bus_stop_count_750m', 'subway_station_count_750m',
        'bus_stop_count_1000m', 'subway_station_count_1000m',
        'distance_to_lead_house', '계약년',
        '올해 이 지역 평균', '올해 이 아파트 평균', '올해 해당 집 유형들의 평균', '평균 부정확도', 'target', 'is_test']
    
    chunk = chunk[select_cols]

    train_chunk = chunk.query('is_test == 0')
    test_chunk = chunk.query('is_test == 1')

    fold_cnt = (len(train_chunk) // 1000)
    fold_cnt = max(fold_cnt, 2)

    tscv = TimeSeriesSplit(n_splits = fold_cnt)

    if region == '중구 입정동':
        one_target = train_chunk['target']
        test_chunk['target'] = one_target
        result = pd.concat([result, test_chunk[['id', 'target']]])
        continue

    if len(test_chunk) == 0:
        regions_have_empty_test.append(region)
        continue

    if len(train_chunk) == 0:
        print("!!!!!!!!")
        print(f"{region}의 학습 데이터가 비어있어요!!!")
        print("!" * 100)
        

    train_chunk.drop(columns = ['is_test', 'id'], axis=1, inplace=True)
    test_chunk.drop(columns = ['is_test'], axis=1, inplace=True)
    
    X = train_chunk.drop(columns = ['target'], axis=1)
    y = train_chunk['target']

    min_rmse = int(1e9)

    model = None
    for fold, (train_index, val_index) in enumerate(tscv.split(X), 1):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        train_data = lgb.Dataset(X_train, label = y_train)
        val_data = lgb.Dataset(X_val, label = y_val, reference = train_data)

        if model :
            model = lgb.train(
                params,
                train_data,
                num_boost_round=1000,
                init_model = model,
                valid_sets = [train_data, val_data],
                callbacks = [lgb.early_stopping(stopping_rounds=5), lgb.log_evaluation(period=-1)],
            )
        else:
            model = lgb.train(
                params,
                train_data,
                num_boost_round=1000,
                valid_sets = [train_data, val_data],
                callbacks = [lgb.early_stopping(stopping_rounds=5), lgb.log_evaluation(period=-1)],
            )

        y_pred = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        min_rmse = min(min_rmse, rmse)

    result_board.append({"지역" : region, "validation RMSE" : min_rmse})

    final_train_data = lgb.Dataset(X, label=y)
    final_model = lgb.train(params,
                            final_train_data,
                            init_model = model,
                            num_boost_round=model.best_iteration,
                            callbacks = [lgb.log_evaluation(period=-1)],
                            )

    test_X = test_chunk.drop(columns=['id', 'target'], axis=1)
    pred = final_model.predict(test_X)

    test_chunk['target'] = pred

    result = pd.concat([result, test_chunk[['id', 'target']]]) 

  0%|          | 0/339 [00:00<?, ?it/s]

Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 12007.9	valid_1's rmse: 11730.9
Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[2000]	training's rmse: 7930.94	valid_1's rmse: 12872.2
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[2981]	training's rmse: 8379.02	valid_1's rmse: 9450.26
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[2982]	training's rmse: 8657.84	valid_1's rmse: 6756.9
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[3619]	training's rmse: 8080.84	valid_1's rmse: 3193.13
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[3728]	training's rmse: 7469.89	valid_1's rmse: 3665.25
Training until validation scores don't improve for 5 rounds
Early sto

  0%|          | 1/339 [00:43<4:02:50, 43.11s/it]

Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 13388.3	valid_1's rmse: 31387.2
Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[2000]	training's rmse: 11620.8	valid_1's rmse: 14946.2
Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[3000]	training's rmse: 8578.24	valid_1's rmse: 15217


  1%|          | 2/339 [00:56<2:24:38, 25.75s/it]

...

Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 6301.36	valid_1's rmse: 5181.08
Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[2000]	training's rmse: 4133.23	valid_1's rmse: 3047.34
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[2001]	training's rmse: 3805.46	valid_1's rmse: 5494.6
Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[3001]	training's rmse: 3774.16	valid_1's rmse: 44483.8


  6%|▌         | 20/339 [16:40<5:09:12, 58.16s/it]

Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 6157.51	valid_1's rmse: 6037.39
Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[2000]	training's rmse: 3346.5	valid_1's rmse: 2598.57
Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[3000]	training's rmse: 2258.86	valid_1's rmse: 1959.33
Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[4000]	training's rmse: 1881.58	valid_1's rmse: 3110.66
Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[5000]	training's rmse: 1908.2	valid_1's rmse: 4599.67
Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[6000]	training's rmse: 2201.25	valid_1's rmse: 8610.09


  6%|▌         | 21/339 [18:08<5:52:14, 66.46s/it]

Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 6830.93	valid_1's rmse: 8946.08
Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[2000]	training's rmse: 4348.76	valid_1's rmse: 3005.14
Training until validation scores don't improve for 5 rounds


In [ ]:
import seaborn as sns

result_df = pd.DataFrame(result_board)
result_df = result_df.sort_values(by='validation RMSE')

sns.barplot(data = result_df, x='지역', y='validation RMSE')

In [ ]:
submit = result.sort_values(by = 'id')

display(result)

nan_val = 81300
submit['target'] = submit['target'].fillna(nan_val)

display(submit)
print(nan_val)
print(submit.isnull().sum())

submit['target'] = submit['target'].astype(int)
submit.to_csv('region_devision_N_mean_features.csv', index=False)

,target
1118822,168205.967724
1118823,238960.136473
1118824,166923.984882
1118825,166930.569634
1118826,168216.869002
...,...
1128089,67953.226014
1128090,66169.778552
1128091,67314.670718
1128092,68219.103592


81300
target    0
dtype: int64
